In [1480]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import datetime
from bs4 import BeautifulSoup
import requests
import string
from concurrent.futures import ThreadPoolExecutor, wait
import copy
from random import uniform
import random

import warnings
warnings.filterwarnings("ignore")


In [1481]:
recg = requests.get('https://www.imdb.com/feature/genre/?ref_=nv_ch_gr')
soupg = BeautifulSoup(recg.content, 'html.parser')

In [1482]:
imgs = soupg.find_all('div', class_= 'image')

In [1483]:
genre_urls = []
for img in imgs:
    draft = img.a['href']
    genre_urls.append(draft[:33] + '/' + draft[33: 74] + '&title_type=movie&ref_=adv_explore_rhs')

In [1484]:
all_urls = []
all_titles = []
all_times = []

#for genre in genre_urls[:-1]:
    #get_genre(genre)

In [1509]:
all_u = []
all_titles = []
rows = []
award_rows = []
current_page = ''
keywords_rows = []
user_agents = [ 
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
	'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
	'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
	'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0'
] 

In [1510]:
def get_genre(genre_url, max_movies):
    
    comedy_awards = pd.DataFrame(columns = ['id', 'awards'])
    comedy_key_words = pd.DataFrame(columns = ['id', 'key_words'])
    comedy = pd.DataFrame(columns = ['id', 'title', 'release_date', 'runtime', 'm_rating', 'imdb_rating', 'num_review', 
                                     'description', 'director', 'writer', 'actors', 'genres' , 'country', 'languages',
                                     'prod_comp', 'budget', 'dom_gross', 'world_gross', 'poster', 'meta_score'])

    print(genre_url)
    start = datetime.datetime.now()
    
    req_cat = requests.get(genre_url)
    soup_cat = BeautifulSoup(req_cat.content, 'html.parser')
    movies = soup_cat.find_all('div', class_='lister-item-image float-left')
    for movie in movies:
        if movie.a['href'] not in all_u:
            all_u.append(movie.a['href'])

    current_page = soup_cat.find('a', class_ = 'lister-page-next next-page')['href']
    
    for num in np.arange(1,max_movies+1):
        print(num)
        get_urls(current_page)
        
    with ThreadPoolExecutor(max_workers = 10) as executor:
        executor.map(get_movie, all_u)
        executor.map(get_awards, all_u)
        executor.map(get_keys, all_u)
        
    for row in rows:
        comedy = comedy.append(row, ignore_index=True)
    for award in award_rows:
        comedy_awards = comedy_awards.append(award, ignore_index = True)
    for keyword in keywords_rows:
        comedy_key_words = comedy_key_words.append(keyword, ignore_index = True)

    comedy1 = pd.merge(comedy_awards, comedy, on = 'id', how = 'outer')
    comedy_df = pd.merge(comedy1, comedy_key_words, on = 'id', how = 'outer').dropna(subset = ['title'])
  
    end = datetime.datetime.now()
    all_times.append(end - start)
    return comedy_df

In [1511]:
def get_urls(page_url):
    req = requests.get('https://www.imdb.com' + page_url)
    soup = BeautifulSoup(req.content, 'html.parser')
    movies = soup.find_all('div', class_='lister-item-image float-left')
    for movie in movies:
        if movie.a['href'] not in all_u:
            all_u.append(movie.a['href'])
    current_page = soup.find('a', class_ = 'lister-page-next next-page')['href']

In [1512]:
def get_movie(movie_url):
    time.sleep(uniform(0,1))
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent} 
    req = requests.get('https://www.imdb.com/' + movie_url, headers = headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    
    try:
        title = soup.find('h1', class_ = 'sc-b73cd867-0 eKrKux').text
    except:
        try: 
            title = soup.find('h1', class_ = 'sc-b73cd867-0 fbOhB').text
        except:
            title = soup.find('h1', class_ = 'sc-b73cd867-0 cAMrQp').text
    
    wr = soup.find_all('ul', role = "presentation", class_ = 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt')
    writers = set([e.text for e in wr[1].findChildren() if ')' not in e.text])
    director = wr[0].text
    description = soup.find('span', class_ = 'sc-16ede01-2 gXUyNh').text
    avg_review = soup.find('span', class_="sc-7ab21ed2-1 jGRxWM").text
    num_review = int(float(soup.find('div', class_="sc-7ab21ed2-3 dPVcnq").text[:-1]) * 1000)
    poster = soup.find('div', class_ = 'ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--baseAlt ipc-media--poster-l ipc-poster__poster-image ipc-media__img').img['src']
    
    try:
        budget = int(soup.find('button', text = 'Budget').findNext('div').text.partition('(')[0][1:].strip().replace(',', ''))
    except:
        budget = 0
        
    try:
        dom_gross = int(soup.find('button', text = 'Gross US & Canada').findNext('div').text.partition('(')[0][1:].strip().replace(',', ''))
    except:
        dom_gross = 0
        
    try:
        world_gross = int(soup.find('button', text = 'Gross worldwide').findNext('div').text.partition('(')[0][1:].strip().replace(',', ''))
    except:
        world_gross = 0
        
    try:
        meta_score = int(soup.find('span', class_ = 'score-meta').text)
    except:
        meta_score = 0
    
    try:
        rating = soup.find_all('a', class_ = 'ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-8c396aa2-1 WIUyh')[1].text
    except:
        rating = 'UNK'
        
    try:
        date = soup.find('a', text = 'Release date').findNext('div').text.partition('(')[0].strip()
    except:
        date = '?'
        
    try:
        country = soup.find('button', text = 'Country of origin').findNext('div').text
    except:
        country = 'unknown'
    
    genres = []
    try:
        gs = soup.find_all('a', class_ ='sc-16ede01-3 bYNgQ ipc-chip ipc-chip--on-baseAlt')
        for g in gs:
            genres.append(g.text)
    except:
        genres = ['none']
       
    all_comps = []
    try:
        comps = soup.find('a', text = 'Production companies').findNext('div').find_all('li')
        for comp in comps:
            all_comps.append(comp.text)
    except:
        all_comps = ['none']
    
    cast = soup.find_all('div', class_ = 'sc-bfec09a1-5 dGCmsL')
    all_cast = []
    for act in cast:
        all_cast.append(act.find_all('div')[1].a['aria-label'])
    
    runtime = soup.find('button', text = 'Runtime').findNext('div').text
    run = int(runtime[0]) * 60
    if len(runtime) > 10:
        run += int(runtime[8])
    
    all_langs = []
    try:
        langs = soup.find('button', text = 'Languages').findNext('div').ul.find_all('li')
        for lang in langs:
            all_langs.append(lang.text)
    except:
        all_langs.append(soup.find('button', text = 'Language').findNext('div').ul.li.text)
    
    rows.append({'id' : int(movie_url[9:-1]), 'title': title, 'writer': writers, 'director': director, 'm_rating': rating,
                 'release_date': date, 'country': country, 'description': description, 'imdb_rating': avg_review, 
                 'num_review': num_review, 'meta_score': meta_score, 'poster': poster, 'languages': all_langs, 'runtime': run,
                 'actors': all_cast, 'genres': genres, 'budget': budget, 'dom_gross': dom_gross, 'world_gross': world_gross,
                 'prod_comp': all_comps})
    print(len(rows))

In [1513]:
def get_keys(movie_url):
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent} 
    
    req = requests.get('https://www.imdb.com/' + movie_url + 'keywords', headers = headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    words = soup.find_all('div', class_ = 'sodatext')
    
    all_words = []
    for word in words:
        all_words.append(word.text.strip())
    
    keywords_rows.append({'id': int(movie_url[9:-1]), 'key_words': all_words})

In [1514]:
def get_awards(movie_url):
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent} 
    
    req = requests.get('https://www.imdb.com/' + movie_url + 'awards', headers = headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    aw = soup.find_all('h3')
    
    all_awards = []
    for d in aw[1:-2]:
        win = ''
        award = ''
        festival = d.text[:-6].strip()

        individuals = d.findNext('table').find_all('tr')
        for ind in individuals:
            win = ind.findNext('b').text[0]
            award = ind.findNext('b').findNext('td').text.partition('\n')[2].partition('\n')[0].strip()

        all_awards.append(win + '/' + award + '/' + festival)
        
    award_rows.append({'id': int(movie_url[9:-1]), 'awards': all_awards})

In [1515]:
co_df = get_genre('https://www.imdb.com/search/title/?title_type=movie&genres=comedy&explore=title_type,genres&ref_=adv_prv', 2115)

https://www.imdb.com/search/title/?title_type=movie&genres=comedy&explore=title_type,genres&ref_=adv_prv
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251

1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039


In [1516]:
co_df

,id,awards,title,release_date,runtime,m_rating,imdb_rating,num_review,description,director,...,genres,country,languages,prod_comp,budget,dom_gross,world_gross,poster,meta_score,key_words
0,9764362.0,[N/Best Original Score - Horror Film/Hollywood...,The Menu,"November 18, 2022",67,R,7.6,16000,A young couple travels to a remote island to e...,Mark Mylod,...,"[Comedy, Horror, Thriller]",United States,"[Spanish, English]","[Alienworx Productions, Alienworx Productions,...",35000000,11326443,17506514,https://m.media-amazon.com/images/M/MV5BMzdjNj...,71,"[restaurant, psychological horror, cooking, ch..."
1,1596342.0,[],Disenchanted,"November 18, 2022",69,PG,5.8,9200,"Fifteen years after her happily ever after, Gi...",Adam Shankman,...,"[Animation, Adventure, Comedy]",United States,[English],"[Walt Disney Pictures, Josephson Entertainment...",0,0,0,https://m.media-amazon.com/images/M/MV5BNDVhYj...,50,"[second part, disney, disney plus, disney prin..."
2,14715170.0,[],Falling for Christmas,"November 10, 2022",63,TV-PG,5.3,11000,"In the days leading up to Christmas, a young a...",Janeen Damian,...,"[Comedy, Romance]",United States,[English],"[Motion Picture Corporation of America (MPCA),...",0,0,0,https://m.media-amazon.com/images/M/MV5BMDNhMW...,39,"[christmas, hotel, love, amnesia, memory loss,..."
3,17220704.0,[],A Christmas Story Christmas,"November 17, 2022",68,PG,7.0,3900,Follows the now-adult Ralphie as he returns to...,Clay Kaytis,...,"[Comedy, Family]",unknown,[English],"[Warner Bros., Legendary Entertainment, Wild W...",0,0,0,https://m.media-amazon.com/images/M/MV5BMjliZj...,56,"[mexico, hungary, christmas, sequel, ralphie p..."
4,12593682.0,"[N/The Movie of 2022/People's Choice Awards, USA]",Bullet Train,"August 5, 2022",127,R,7.3,183000,Five assassins aboard a swiftly-moving bullet ...,David Leitch,...,"[Action, Comedy, Thriller]",unknown,"[English, Japanese, Spanish, Russian]","[87North, CTB Inc., Hill District Media]",85900000,103368602,239268602,https://m.media-amazon.com/images/M/MV5BMDU2Zm...,49,"[train, assassin, russian, briefcase, fight on..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1570728,NaN,"Crazy, Stupid, Love.","July 29, 2011",68,PG-13,7.4,522000,A middle-aged husband's life changes dramatica...,Glenn FicarraJohn Requa,...,"[Comedy, Drama, Romance]",United States,[English],[none],50000000,84379584,145079584,https://m.media-amazon.com/images/M/MV5BMTg2Mj...,68,"[divorce, marriage, unfaithfulness, adultery, ..."
96,100405,NaN,Pretty Woman,"March 23, 1990",69,R,7.1,326000,A man in a legal but hurtful business needs an...,Garry Marshall,...,"[Comedy, Romance]",United States,[English],"[Touchstone Pictures, Silver Screen Partners IV]",14000000,178406268,463406268,https://m.media-amazon.com/images/M/MV5BNjk2OD...,51,"[prostitute, hotel, class differences, busines..."
97,2278388,NaN,The Grand Budapest Hotel,"March 28, 2014",69,R,8.1,806000,A writer encounters the owner of an aging high...,Wes Anderson,...,"[Adventure, Comedy, Crime]",unknown,"[English, French, German]","[Searchlight Pictures, Indian Paintbrush, Stud...",25000000,59301324,173082189,https://m.media-amazon.com/images/M/MV5BMzM5Nj...,88,"[hotel, lobby boy, first person narration, pai..."
98,11252248,NaN,Dog,"February 18, 2022",61,PG-13,6.5,34000,Two former Army Rangers are paired against the...,Reid CarolinChanning Tatum,...,"[Comedy, Drama]",United States,[English],"[Metro-Goldwyn-Mayer (MGM), CAA Media Finance,...",15000000,61778069,84774243,https://m.media-amazon.com/images/M/MV5BYjE0ZD...,61,"[dog, dog movie, funeral, pretending to be bli..."


In [1517]:
all_times[-1]

datetime.timedelta(seconds=983, microseconds=842202)

In [1427]:
#url = '/title/tt9764362/'
url = '/title/tt1596342/'
#url = '/title/tt14715170/'
#url = '/title/tt0095016/'

req1 = requests.get('https://www.imdb.com/' + url, headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"})
soup = BeautifulSoup(req1.content, 'html.parser')


In [1035]:
awards = get_awards(url + 'awards')

In [1036]:
awards

[]

In [ ]:
def get_movie(movie_url):
    req = requests.get('https://www.imdb.com/' + movie_url, headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"})
    soup = BeautifulSoup(req.content, 'html.parser')
    
    title = ''
    try:
        title = soup.find('h1', class_ = 'sc-b73cd867-0 eKrKux').text
    except:
        try: 
            title = soup.find('h1', class_ = 'sc-b73cd867-0 fbOhB').text
        except:
            title = soup.find('h1', class_ = 'sc-b73cd867-0 cAMrQp').text
    print(title)
    wr = soup.find_all('ul', role = "presentation", class_ = 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt')
    writers = set([e.text for e in wr[1].findChildren() if ')' not in e.text])
    director = wr[0].text
    rating = soup.find_all('a', class_ = 'ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-8c396aa2-1 WIUyh')[1].text
    date = soup.find('a', text = 'Release date').findNext('div').text.partition('(')[0].strip()
    country = soup.find('button', text = 'Country of origin').findNext('div').text
    description = soup.find('span', class_ = 'sc-16ede01-2 gXUyNh').text
    avg_review = soup.find('span', class_="sc-7ab21ed2-1 jGRxWM").text
    num_review = int(float(soup.find('div', class_="sc-7ab21ed2-3 dPVcnq").text[:-1]) * 1000)
    poster = soup.find('div', class_ = 'ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--baseAlt ipc-media--poster-l ipc-poster__poster-image ipc-media__img').img['src']
    budget = int(soup.find('button', text = 'Budget').findNext('div').text.partition('(')[0][1:].strip().replace(',', ''))
    dom_gross = int(soup.find('button', text = 'Gross US & Canada').findNext('div').text.partition('(')[0][1:].strip().replace(',', ''))
    world_gross = int(soup.find('button', text = 'Gross worldwide').findNext('div').text.partition('(')[0][1:].strip().replace(',', ''))
    meta = int(soup.find('span', class_ = 'score-meta').text)
    
    runtime = soup.find('button', text = 'Runtime').findNext('div').text
    run = int(runtime[0]) * 60
    if len(runtime) > 10:
        run += int(runtime[8])
    
    gs = soup.find_all('a', class_ ='sc-16ede01-3 bYNgQ ipc-chip ipc-chip--on-baseAlt')
    genres = []
    for g in gs:
        genres.append(g.text)
    
    cast = soup.find_all('div', class_ = 'sc-bfec09a1-5 dGCmsL')
    all_cast = []
    for act in cast:
        all_cast.append(act.find_all('div')[1].a['aria-label'])
    
    all_langs = []
    try:
        langs = soup.find('button', text = 'Languages').findNext('div').ul.find_all('li')
        for lang in langs:
            all_langs.append(lang.text)
    except:
        all_langs.append(soup.find('button', text = 'Language').findNext('div').ul.li.text)
        
    comps = soup.find('a', text = 'Production companies').findNext('div').find_all('li')
    all_comps = []
    for comp in comps:
        all_comps.append(comp.text)
    
    key_words = get_keys(movie_url + 'keywords')
    awards = get_awards(movie_url + 'awards')
    return ({'id' : int(movie_url[9:-1]), 'title': title, 'director': director, 'writer':writers, 'm_rating': rating,
                 'release_date': date, 'country': country, 'actors': all_cast, 'languages': all_langs, 'runtime': run,
                 'prod_comp': all_comps, 'description': description, 'num_review': num_review, 'imdb_rating': avg_review,
                 'genres': genres, 'poster': poster, 'budget': budget, 'dom_gross': dom_gross, 'world_gross': world_gross,
                 'meta': meta, 'awards': awards, 'key_words': key_words})